In [8]:
import os
import pandas as pd

os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data1 = pd.read_csv('0726_masking_labeling_data_0.csv')
data2 = pd.read_csv('0726_masking_labeling_data_1.csv')

In [9]:
data1 = data1.rename(columns={'text_morphed_masked':'text_morphed_masked_'})
data2 = data2.rename(columns={'text_morphed_masked':'text_morphed_masked_'})
data1 = data1.rename(columns={'저격성 민원':'저격성민원_'})
data2 = data2.rename(columns={'저격성 민원':'저격성민원_'})
selected_columns1 = ['text_morphed_masked_', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성민원_']
data1 = data1[selected_columns1]
data2 = data2[selected_columns1]
# data1 = data1[:366]

In [3]:

# 데이터 병합
data3 = pd.concat([data1, data2])

In [10]:
data2['저격성민원_'].value_counts()

저격성민원_
0    229
1    137
Name: count, dtype: int64

# 학습

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib


# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data1 = pd.read_csv('0726_masking_labeling_data_0.csv')
data2 = pd.read_csv('0726_masking_labeling_data_1.csv')

data1 = data1.rename(columns={'text_morphed':'text_morphed_'})
data2 = data2.rename(columns={'text_morphed':'text_morphed_'})


selected_columns1 = ['title','text_morphed_', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성 민원']

target = selected_columns1[6]

data1 = data1[selected_columns1]
data2 = data2[selected_columns1]
data1 = data1[300:437]                    #라벨 데이터 개수만큼 설정
data2 = data2[data2[target] == 1]   #라벨 컬럼 설정

data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
data_contents = data3['text_morphed_'] + ' ' + data3['title']
data_labeling = data3[f'{target}']    #라벨 컬럼 설정

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_contents, data_labeling, stratify=data_labeling, test_size=0.2, random_state=42)

# 파이프라인 정의
pipelines = {

    'extra_trees': Pipeline([
        ('vect', TfidfVectorizer()),
        ('et', ExtraTreesClassifier())
    ]),

    'ridge_classifier': Pipeline([
        ('vect', TfidfVectorizer()),
        ('ridge', RidgeClassifier())
    ]),

    'logistic_regression': Pipeline([
        ('vect', TfidfVectorizer()),
        ('lr', LogisticRegression())
    ]),

    'naive_bayes': Pipeline([
        ('vect', TfidfVectorizer()),
        ('nb', MultinomialNB())
    ]),

    'svm': Pipeline([
        ('vect', TfidfVectorizer()),
        ('svm', SVC())
    ]),


    'hard_model' : Pipeline([
        ('vect', TfidfVectorizer()),
      ('hard_model',VotingClassifier([('LR',LogisticRegression()),('ridge',RidgeClassifier()),('svm',SVC()),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='hard'))

    ]),
    'soft_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('soft_model',VotingClassifier([('LR',LogisticRegression()),('svm',SVC(probability=True)),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='soft'))

    ])
    

}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")

    # 텍스트 데이터를 TF-IDF 벡터로 변환
    X_train_tfidf = pipeline.named_steps['vect'].fit_transform(X_train)
    X_test_tfidf = pipeline.named_steps['vect'].transform(X_test)
    
    # 차원 출력
    print(f"{model_name} - X_train shape: {X_train_tfidf.shape}")
    print(f"{model_name} - X_test shape: {X_test_tfidf.shape}")
    
    # 모델 훈련
    pipeline.fit(X_train, y_train)
    
    # 교차검증 점수 계산
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    # 성능 출력
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    print(f"{model_name} Train set score: {train_score:.4f}")
    print(f"{model_name} Test set score: {test_score:.4f}")
    print(f"{model_name} Cross-validation scores: {cv_scores}")
    print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")
    
    # 모델 저장
    model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'{target}_{model_name}_tfidf_nonemasking_model.pkl')
    joblib.dump(pipeline, model_save_path)

print("All models have been trained and saved.")

Training extra_trees...
extra_trees - X_train shape: (114, 3397)
extra_trees - X_test shape: (29, 3397)
extra_trees Train set score: 1.0000
extra_trees Test set score: 0.9655
extra_trees Cross-validation scores: [0.95652174 0.95652174 0.95652174 0.95652174 0.95454545]
extra_trees Cross-validation mean score: 0.9561

Training ridge_classifier...
ridge_classifier - X_train shape: (114, 3397)
ridge_classifier - X_test shape: (29, 3397)
ridge_classifier Train set score: 1.0000
ridge_classifier Test set score: 0.9655
ridge_classifier Cross-validation scores: [0.95652174 0.95652174 0.95652174 0.95652174 0.95454545]
ridge_classifier Cross-validation mean score: 0.9561

Training logistic_regression...
logistic_regression - X_train shape: (114, 3397)
logistic_regression - X_test shape: (29, 3397)
logistic_regression Train set score: 0.9561
logistic_regression Test set score: 0.9655
logistic_regression Cross-validation scores: [0.95652174 0.95652174 0.95652174 0.95652174 0.95454545]
logistic_reg

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline
import joblib
from sklearn.naive_bayes import MultinomialNB

# XGBoost, LightGBM, CatBoost import
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import os
import pandas as pd

os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data1 = pd.read_csv('masking_labeling_data.csv')
data2 = pd.read_csv('masking_labeling_data2.csv')

data1 = data1.rename(columns={'text_morphed_masked':'text_morphed_masked_'})
data2 = data2.rename(columns={'text_morphed_masked':'text_morphed_masked_'})
data1 = data1.rename(columns={'저격성 민원':'저격성민원_'})
data2 = data2.rename(columns={'저격성 민원':'저격성민원_'})

selected_columns1 = ['text_morphed_masked_', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성민원_']
data1 = data1[selected_columns1]
data2 = data2[selected_columns1]
data1 = data1[:40]
data2 = data2[data2.저격성민원_ == 1]                #독립변수 설정

data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
data_contents = data3.iloc[:, 0]
data_labeling = data3.loc[:, '저격성민원_']             #독립변수 설정

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_contents, data_labeling, stratify=data_labeling, test_size=0.2)

# 파라미터 그리드 정의
param_grids = {
    'logistic_regression': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'lr__C': [0.001, 0.01, 0.1, 1, 10],
        'lr__solver': ['liblinear', 'lbfgs']
    },
    'svm': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'svm__C': [0.001, 0.01, 0.1, 1, 10],
        'svm__kernel': ['linear', 'rbf']
    },
    'random_forest': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'rf__n_estimators': [50, 100, 200],
        'rf__max_depth': [None, 10, 20, 30]
    },
    'gradient_boosting': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'gb__n_estimators': [50, 100, 200],
        'gb__learning_rate': [0.01, 0.1, 0.2],
        'gb__max_depth': [3, 5, 7]
    },
    'ada_boost': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'ada__n_estimators': [50, 100, 200],
        'ada__learning_rate': [0.001, 0.01, 0.1]
    },
    'xgboost': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 7]
    },
    'lightgbm': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'lgb__n_estimators': [50, 100, 200],
        'lgb__learning_rate': [0.01, 0.1, 0.2],
        'lgb__max_depth': [3, 5, 7]
    },
    'catboost': {
        'vect__max_features': [10000, 20000, 30000],
        'pca__n_components': [100, 500, 1000],  # TruncatedSVD의 n_components 추가
        'cat__iterations': [50, 100, 200],
        'cat__learning_rate': [0.01, 0.1, 0.2],
        'cat__depth': [3, 5, 7]
    }
}

# 파이프라인 정의
pipelines = {
    'logistic_regression': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('lr', LogisticRegression())
    ]),
    'svm': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('svm', SVC())
    ]),
    'random_forest': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('rf', RandomForestClassifier())
    ]),
    'gradient_boosting': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('gb', GradientBoostingClassifier())
    ]),
    'ada_boost': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('ada', AdaBoostClassifier())
    ]),
    'xgboost': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('xgb', xgb.XGBClassifier(eval_metric='mlogloss'))
    ]),
    'lightgbm': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('lgb', lgb.LGBMClassifier())
    ]),
    'catboost': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD()),  # TruncatedSVD의 n_components는 그리드 서치에서 조정됨
        ('cat', cb.CatBoostClassifier(learning_rate=0.1, iterations=100, depth=6, verbose=0))
    ])
}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    
    # 그리드 서치 정의
    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
    
    # 모델 훈련
    grid_search.fit(X_train, y_train)
    
    # 최적의 파라미터와 성능 출력
    print(f"\n{model_name} Best parameters found: {grid_search.best_params_}")
    print(f"{model_name} Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # 최적의 모델로 테스트 데이터 평가
    test_score = grid_search.score(X_test, y_test)
    print(f"{model_name} Test set score: {test_score:.4f}\n")
    
    # 모델 저장
    model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'저격성민원_{model_name}_model_grid_search.pkl')     #독립변수 설정
    joblib.dump(grid_search, model_save_path)

print("All models have been trained and saved.")


Training logistic_regression...
Fitting 3 folds for each of 90 candidates, totalling 270 fits

logistic_regression Best parameters found: {'lr__C': 1, 'lr__solver': 'liblinear', 'pca__n_components': 100, 'vect__max_features': 10000}
logistic_regression Best cross-validation score: 0.8766
logistic_regression Test set score: 0.8750

Training svm...
Fitting 3 folds for each of 90 candidates, totalling 270 fits

svm Best parameters found: {'pca__n_components': 100, 'svm__C': 10, 'svm__kernel': 'rbf', 'vect__max_features': 10000}
svm Best cross-validation score: 0.7352
svm Test set score: 0.7500

Training random_forest...
Fitting 3 folds for each of 108 candidates, totalling 324 fits

random_forest Best parameters found: {'pca__n_components': 500, 'rf__max_depth': 20, 'rf__n_estimators': 50, 'vect__max_features': 30000}
random_forest Best cross-validation score: 0.8608
random_forest Test set score: 0.7500

Training gradient_boosting...
Fitting 3 folds for each of 243 candidates, totalling 7

c:\Users\USER\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



xgboost Best parameters found: {'pca__n_components': 100, 'vect__max_features': 30000, 'xgb__learning_rate': 0.2, 'xgb__max_depth': 3, 'xgb__n_estimators': 200}
xgboost Best cross-validation score: 0.8449
xgboost Test set score: 0.8750

Training lightgbm...
Fitting 3 folds for each of 243 candidates, totalling 729 fits
[LightGBM] [Info] Number of positive: 32, number of negative: 32
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1452
[LightGBM] [Info] Number of data points in the train set: 64, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

In [66]:
data_contents

0      안녕 하 관리자 `지역` 문화센터 인근 학교 재직 교사 `지역` 문화센터 근방 신호...
1      안녕 하 `지역` 아이 주민 에 인도 파손 되 넘어지 경험 있 인도 파손 부분 인도...
2      `지역` `지역` `지역` 차선 도로 파손 되 매우 크 불편 겪 사진 첨부 지하철 ...
3      `지역` 교차로 비보 호 좌회전 신호 있 좌회전 신호일 때 좌회전 도로 횡단보도 신...
4      안녕 하 코로나 도둑 맞 같 현실 모든 가족 너무 화 세상에 있 답답 일단 국민 신...
                             ...                        
192    `지역` 빌 쓰레기 처리 대하 건의 사람 알 빌라 주민 만나 옮기 주시 하 궁 나 ...
193    추석연 휴 친정 이틀 `지역` 빌라 차이 아직 덥 때 문 열 미치 알 사람 버리 쓰...
194                 지나 달 차례 문의 드리 아직 방지 턱 설치 안 되 있 지나 치이
195    군수 건축 과장 안녕 하 지나 일요일 오전 군청 발주 하 `지역` 농공 단지 앞 하...
196    군도 호선 미개 설 공 촉구 건 대하 부여군 건설과 도로계 대답 시원 보충 질문 존...
Name: text_morphed_masked_, Length: 395, dtype: object

In [14]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline
import joblib
import rhinoMorph

# XGBoost, LightGBM, CatBoost import
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# 경로 설정
data_path = 'C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/라벨링 데이터/'
os.chdir(data_path)

# 데이터 읽기
data1 = pd.read_csv('공격성_민원_0.csv')
data2 = pd.read_csv('공격성 민원 모음.csv')

# 데이터 전처리
data2 = data2.drop('Unnamed: 0', axis=1)
data2.rename(columns={"민원내용": 'title_and_contents'}, inplace=True)
data1.rename(columns={"저격성 민원": '저격성민원'}, inplace=True)

# 데이터 병합
data3 = pd.concat([data1, data2])
data4 = data3[['title_and_contents', 'aggr']].reset_index(drop=True)

# 텍스트와 라벨 분리
data_contents = data4['title_and_contents']
data_labeling = data4['aggr']

# 형태소 분석
rn = rhinoMorph.startRhino()

def morph_and_label(contents, labels):
    location_data = pd.read_csv('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/행정구역 데이터/file/행정구역명리스트.csv')
    location_list = list(location_data.loc[:, 'combined'])

    rn = rhinoMorph.startRhino()

    morphed_data_list = []
    for data_each in data_contents:
        morphed_data_each, poses = rhinoMorph.wholeResult_list(rn, str(data_each),pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
        # print(morphed_data_each)
        # print(poses)
        # 형태소 분석된 데이터에서 NNP에 대해 이름 정규 표현식 적용 및 지역명 체크
        for j, item in enumerate(poses):
            if item == 'NNP':
                word = morphed_data_each[j]
                
                # 지역명 리스트에 있는지 체크
                if word in location_list:
                    morphed_data_each[j] = '`지역`'
            
        joined_data_each = ' '.join(morphed_data_each)
        if joined_data_each:
            morphed_data_list.append(joined_data_each)
        else : 
            morphed_data_list.append(' ')

morphed_data = morph_and_label(data_contents, data_labeling)

# 데이터프레임 생성
morphed_df = pd.DataFrame(morphed_data, columns=['text', 'label'])

# 데이터 저장
morphed_file = 'ratings_morphed.txt'
morphed_df.to_csv(morphed_file, index=False, encoding='cp949')

# 데이터 로드
data = pd.read_csv(morphed_file, encoding='cp949')

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], stratify=data['label'], test_size=0.2)

# 파라미터 그리드 정의
param_grids = {
    'logistic_regression': {
        'vect__max_features': [10000, 20000, 30000],
        'lr__C': [0.001, 0.01, 0.1, 1, 10],
        'lr__solver': ['liblinear', 'lbfgs']
    },
    'svm': {
        'vect__max_features': [10000, 20000, 30000],
        'svm__C': [0.001, 0.01, 0.1, 1, 10],
        'svm__kernel': ['linear', 'rbf']
    },
    'random_forest': {
        'vect__max_features': [10000, 20000, 30000],
        'rf__n_estimators': [50, 100, 200],
        'rf__max_depth': [None, 10, 20, 30]
    },
    'gradient_boosting': {
        'vect__max_features': [10000, 20000, 30000],
        'gb__n_estimators': [50, 100, 200],
        'gb__learning_rate': [0.01, 0.1, 0.2],
        'gb__max_depth': [3, 5, 7]
    },
    'naive_bayes': {
        'vect__max_features': [10000, 20000, 30000]
    },
    'ada_boost': {
        'vect__max_features': [10000, 20000, 30000],
        'ada__n_estimators': [50, 100, 200],
        'ada__learning_rate': [0.001, 0.01, 0.1]
    },
    'xgboost': {
        'vect__max_features': [10000, 20000, 30000],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 7]
    },
    'lightgbm': {
        'vect__max_features': [10000, 20000, 30000],
        'lgb__n_estimators': [50, 100, 200],
        'lgb__learning_rate': [0.01, 0.1, 0.2],
        'lgb__max_depth': [3, 5, 7]
    },
    'catboost': {
        'vect__max_features': [10000, 20000, 30000],
        'cat__iterations': [50, 100, 200],
        'cat__learning_rate': [0.01, 0.1, 0.2],
        'cat__depth': [3, 5, 7]
    }
}

# 파이프라인 정의
pipelines = {
    'logistic_regression': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('lr', LogisticRegression())
    ]),
    'svm': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('svm', SVC())
    ]),
    'random_forest': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('rf', RandomForestClassifier())
    ]),
    'gradient_boosting': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('gb', GradientBoostingClassifier())
    ]),
    'ada_boost': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('ada', AdaBoostClassifier())
    ]),
    'xgboost': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('xgb', xgb.XGBClassifier(eval_metric='mlogloss'))
    ]),
    'lightgbm': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('lgb', lgb.LGBMClassifier())
    ]),
    'catboost': Pipeline([
        ('vect', CountVectorizer()),
        ('pca', TruncatedSVD(n_components=1000)),
        ('cat', cb.CatBoostClassifier(learning_rate=0.1, iterations=100, depth=6, verbose=0))
    ])
}

# 모델 훈련 및 평가
model_save_path = 'C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/'
os.chdir(model_save_path)

for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    
    # 그리드 서치 정의
    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
    
    # 모델 훈련
    grid_search.fit(X_train, y_train)
    
    # 최적의 파라미터와 성능 출력
    print(f"\n{model_name} Best parameters found: {grid_search.best_params_}")
    print(f"{model_name} Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # 최적의 모델로 테스트 데이터 평가
    test_score = grid_search.score(X_test, y_test)
    print(f"{model_name} Test set score: {test_score:.4f}\n")
    
    # 모델 저장
    joblib.dump(grid_search, f'{model_name}_model_grid_search.pkl')


filepath:  C:\Users\USER\anaconda3\Lib\site-packages
classpath:  C:\Users\USER\anaconda3\Lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
filepath:  C:\Users\USER\anaconda3\Lib\site-packages
classpath:  C:\Users\USER\anaconda3\Lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.